In [ ]:
# Import used packages
import numpy as np
import matplotlib.pyplot as plt
import random
import networkx as nx

In [ ]:
# Create network and startsituation (and colors)
def startcolor(nodes, exposedPerc, radius, color):
    Network = nx.random_geometric_graph(nodes, radius)
    Situation = []
    Colorlist = []
    for i in range(nodes):
        randomN = random.uniform(0, 1)
        if randomN < exposedPerc:
            Situation.append(3)
            Colorlist.append(color[3])
        else:
            Situation.append(0)
            Colorlist.append(color[0])
    return Network, Situation, Colorlist

In [ ]:
## SEIR with color
# 0 for S, 1 for I, 2 for R, 3 for E
def updateSEIRcolor(y, beta, delta, gamma, alpha, Network, color):
    yNew = []
    Colorlist = []
    for i in range(len(Network)):
        if y[i] == 0:
            Connected = Network[i]
            Phealthy = 1
            for j in Connected:
                if y[j] == 1:
                    Phealthy *= (1 - beta)
                elif y[j] == 3:
                    Phealthy *= (1 - delta)
            PSick = 1 - Phealthy
            randomN = random.uniform(0, 1)
            if randomN < PSick:
                yNew.append(3)
                Colorlist.append(color[3])
            else:
                yNew.append(y[i])
                Colorlist.append(color[y[i]])
        elif y[i] == 1:
            randomN = random.uniform(0, 1)
            if randomN < gamma:
                yNew.append(2)
                Colorlist.append(color[2])
            else:
                yNew.append(y[i])
                Colorlist.append(color[y[i]])
        elif y[i] == 3:
            randomN = random.uniform(0, 1)
            if randomN < alpha:
                yNew.append(1)
                Colorlist.append(color[1])
            else:
                yNew.append(y[i])
                Colorlist.append(color[y[i]])
        else:
            yNew.append(y[i])
            Colorlist.append(color[y[i]])
    return yNew, Colorlist

def spreaddiseaseSEIRcolor(NLoops, StartSituation, beta, delta, gamma, alpha, Network, color, plotnetworks):
    yList = [StartSituation]
    yNew = StartSituation
    pos = nx.get_node_attributes(Network, 'pos')
    for i in range(NLoops):
        y = yNew
        yNew, Colorlist = updateSEIRcolor(y, beta, delta, gamma, alpha, Network, color)
        yList.append(yNew)
        
        if plotnetworks:
            plt.figure(figsize = (8, 8))
            nx.draw_networkx_edges(Network, pos, alpha= 0.4)
            nx.draw_networkx_nodes(Network, pos, node_size= 100, node_color= Colorlist)
            plt.axis('off')
            plt.show()
    return yList

def countingSEIR(situations):
    S = []
    I = []
    R = []
    E = []
    for i in range(len(situations)):
        S.append(np.bincount(situations[i], minlength = 4)[0])
        I.append(np.bincount(situations[i], minlength = 4)[1])
        R.append(np.bincount(situations[i], minlength = 4)[2])
        E.append(np.bincount(situations[i], minlength = 4)[3])
    return S, I, R, E

def everythingSEIRcolor(nodes, sickPerc, radius, NLoops, beta, delta, gamma, alpha, color, plotnetworks):
    Network, StartSituation, Colorlist = startcolor(nodes, exposedPerc, radius, color)
    
    if plotnetworks:
        pos = nx.get_node_attributes(Network, 'pos')
        plt.figure(figsize = (8, 8))
        nx.draw_networkx_edges(Network, pos, alpha= 0.4)
        nx.draw_networkx_nodes(Network, pos, node_size= 100, node_color= Colorlist)
        plt.axis('off')
        plt.show()
    
    situations = spreaddiseaseSEIRcolor(NLoops, StartSituation, beta, delta, gamma, alpha, Network, color, plotnetworks)
    
    S, I, R, E = countingSEIR(situations)
    time = np.arange(0, NLoops + 1, 1)
    Fig_Data = plt.figure()

    plt.plot(time, S, label = 'S', color= color[0])
    plt.plot(time, I, label = 'I', color= color[1])
    plt.plot(time, R, label = 'R', color= color[2])
    plt.plot(time, E, label = 'E', color= color[3])

    plt.title('', size= 20)
    plt.ylabel('', size= 20)
    plt.xlabel('', size= 20)
    plt.legend(fontsize= 12)
    plt.tick_params(axis= 'both', which= 'major', labelsize= 14)
    plt.grid()

    plt.show()
    return Network, S, I, R, E

def averageSEIR(nodes, sickPerc, radius, NLoops, beta, delta, gamma, alpha, N, color):
    time = np.arange(0, NLoops + 1, 1)
    AverageS = np.zeros(NLoops + 1)
    AverageI = np.zeros(NLoops + 1)
    AverageR = np.zeros(NLoops + 1)
    AverageE = np.zeros(NLoops + 1)
    for i in range(N):
        Network, StartSituation, Colorlist = startcolor(nodes, exposedPerc, radius, color)
        situations = spreaddiseaseSEIRcolor(NLoops, StartSituation, beta, delta, gamma, alpha, Network, color, plotnetworks = False)
        S, I, R, E = countingSEIR(situations)
        AverageS += S
        AverageI += I
        AverageR += R
        AverageE += E
    AverageS = AverageS/N
    AverageI = AverageI/N
    AverageR = AverageR/N
    AverageE = AverageE/N
    
    Fig_Data = plt.figure()

    plt.plot(time, AverageS, label = 'S', color= color[0])
    plt.plot(time, AverageI, label = 'I', color= color[1])
    plt.plot(time, AverageR, label = 'R', color= color[2])
    plt.plot(time, AverageE, label = 'E', color= color[3])

    plt.title('', size= 20)
    plt.ylabel('', size= 20)
    plt.xlabel('', size= 20)
    plt.legend(fontsize= 12)
    plt.tick_params(axis= 'both', which= 'major', labelsize= 14)
    plt.grid()
    
    plt.show()
    return AverageS, AverageI, AverageR, AverageE

In [ ]:
# 1 network for SEIR
nodes = 100
exposedPerc = 0.1
radius = 0.15
beta = 0.2
delta = 0.2
gamma = 0.02
alpha = 0.2
NLoops = 10
color = ['palegreen', 'tomato', 'deepskyblue', 'orchid']
plotnetworks = False

Network, S, I, R, E = everythingSEIRcolor(nodes, exposedPerc, radius, NLoops, beta, delta, gamma, alpha, color, plotnetworks)

In [ ]:
# Average over N networks for SEIR
N = 100
nodes = 100
exposedPerc = 0.1
radius = 0.15
beta = 0.2
delta = 0.2
gamma = 0.02
alpha = 0.2
NLoops = 200
color = ['palegreen', 'tomato', 'deepskyblue', 'orchid']

AverageS, AverageI, AverageR, AverageE = averageSEIR(nodes, exposedPerc, radius, NLoops, beta, delta, gamma, alpha, N, color)